<a href="https://colab.research.google.com/github/johncoder-30/Pytorch-Deep_learning/blob/master/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
import re
from collections import Counter
from nltk.corpus import stopwords
import torch.nn as nn
import numpy as np
from torch.utils.data import TensorDataset,DataLoader

In [ ]:
# import nltk
# nltk.download('stopwords')

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU is available


In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
# ! kaggle datasets list
# ! kaggle competitions download <name-of-competition>
! kaggle datasets download lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
! unzip imdb-dataset-of-50k-movie-reviews.zip


mkdir: cannot create directory ‘/root/.kaggle’: File exists
 78% 20.0M/25.7M [00:01<00:00, 14.9MB/s]
100% 25.7M/25.7M [00:01<00:00, 23.1MB/s]
Archive:  imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [ ]:
data = pd.read_csv('./IMDB Dataset.csv')

# print(data.head(5))
# print(len(data))
x, y = data['review'].values, data['sentiment'].values
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y,shuffle=True)
# unique, counts = np.unique(y_train, return_counts=True)
# print(dict(zip(unique, counts)))
# unique, counts = np.unique(y_test, return_counts=True)
# print(dict(zip(unique, counts)))

In [ ]:
def preprocess(s):
    s = re.sub(r'[^\w\s]', '', s)
    s = re.sub(r'[\d+]', '', s)
    s = re.sub(r'<[a-z]*>', '', s)
    return s


# print(preprocess('asdf adsf4# <as> dfse3  sgd'))
# stop_words = set(stopwords.words('english'))

words = []
for i in x_train:
    i = re.sub(r'\<[^>]*\>', '', i)
    i = re.sub(r'\s+', ' ', i)
    for j in i.split():
        word_ = preprocess(j.lower())
        # if (word_ not in stop_words) and (word_ != ''):
        #     words.append(word_)
        if word_ != '':
            words.append(word_)

corpus = Counter(words)
corpus = sorted(corpus, reverse=True, key=corpus.get)[:998]
corpus = ['_pad', '_unk'] + corpus
# print(corpus)
# onehot_dict = {w: i + 1 for i, w in enumerate(corpus)}
# print(len(onehot_dict))
word2idx = {w: i for i, w in enumerate(corpus)}
idx2word = {i: w for i, w in enumerate(corpus)}

In [ ]:
def sentence_encoder(sen):
    y = []
    for i in sen:
        x = [word2idx[preprocess(word)] if preprocess(word) in word2idx else 0 for word in i.lower().split()]
        y.append(x)
    return y

def pading(sen, length):
    x = np.zeros((len(sen), length))
    for i, line in enumerate(sen):
        x[i, -len(line):] = np.array(line)[:length]
    return x

x_train_encoded=sentence_encoder(x_train)
x_train_paded=pading(x_train_encoded,200)
y_train_encoded=[1 if yy =='positive' else 0 for yy in y_train]
y_train=np.array(y_train_encoded)
unique, counts = np.unique(y_train, return_counts=True)
print(dict(zip(unique, counts)))
# print(y_train_encoded)
train_dataset = TensorDataset(torch.from_numpy(x_train_paded),torch.from_numpy(y_train))
train_loader = DataLoader(train_dataset,batch_size=64)

x_test_encoded=sentence_encoder(x_test)
x_test_paded=pading(x_test_encoded,200)
y_test_encoded=[1 if yy =='positive' else 0 for yy in y_test]
y_test=np.array(y_test_encoded)
unique, counts = np.unique(y_test, return_counts=True)
print(dict(zip(unique, counts)))
test_dataset = TensorDataset(torch.from_numpy(x_test_paded),torch.from_numpy(y_test))
test_loader = DataLoader(test_dataset,batch_size=64)

{0: 20000, 1: 20000}
{0: 5000, 1: 5000}


In [ ]:
class Sentiment(nn.Module):
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        super(Sentiment,self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self,x,hid):
        batch_size = x.size(0)
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hid)
        lstm_out = lstm_out[:,-1,:]
        
        out = self.dropout(lstm_out)
        out = self.fc(out)
        out = self.sigmoid(out)
        return out, hidden

In [ ]:
vocab_size=len(word2idx)
output_size = 1
embedding_dim = 100
hidden_dim = 50
n_layers = 2
batch_size = 64

model=Sentiment(vocab_size,output_size,embedding_dim,hidden_dim,n_layers)
model.to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [ ]:
model.train()
for e in range(10):
    for i,(review,label) in enumerate(train_loader):
        batch_size=review.shape[0]
        h0=torch.zeros(n_layers,batch_size,hidden_dim).to(device)
        c0=torch.zeros(n_layers,batch_size,hidden_dim).to(device)
        h=(h0,c0)
        model_out,h=model(review.long().to(device),h)
        loss=criterion(model_out.squeeze(),label.float().to(device))
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(e,loss.item())    

0 0.5895379781723022
1 0.43732044100761414
2 0.7048697471618652
3 0.5946943759918213
4 0.33692270517349243
5 0.3034355044364929
6 0.24483713507652283
7 0.21968568861484528
8 0.20978952944278717
9 0.18985062837600708


In [ ]:
num_correct = 0
model.eval()
for review,label in test_loader:
    batch_size=review.shape[0]
    h0=torch.zeros(n_layers,batch_size,hidden_dim).to(device)
    c0=torch.zeros(n_layers,batch_size,hidden_dim).to(device)
    h=(h0,c0)
    pred,h=model(review.long().to(device),h)
    pred=pred.round()
    correct_tensor = pred.cpu().eq(label.float().view_as(pred.to('cpu')))
    correct = np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)
print(num_correct)

test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}%".format(test_acc*100))



8447
Test accuracy: 84.470%
